<a href="https://colab.research.google.com/github/Meenakshi2434/Machine-Learning/blob/main/Apriori_Retail_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
df= pd.read_csv('/content/Online Retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df['Description']=df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo']=df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
175967,551995,22980,PANTRY SCRUBBING BRUSH,1,2011-05-05 15:42:00,3.29,15287.0,United Kingdom
175968,551995,22982,PANTRY PASTRY BRUSH,1,2011-05-05 15:42:00,2.46,15287.0,United Kingdom
175969,551995,22983,CARD BILLBOARD FONT,1,2011-05-05 15:42:00,0.83,15287.0,United Kingdom
175970,551995,22993,SET OF 4 PANTRY JELLY MOULDS,3,2011-05-05 15:42:00,2.46,15287.0,United Kingdom


In [5]:
basket = (df[df['Country'] == 'France']).groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [6]:
basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE WOODLAND,3D HEARTS HONEYCOMB PAPER GARLAND,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,60 TEATIME FAIRY CAKE CASES,72 SWEETHEART FAIRY CAKE CASES,ABC TREASURE BOOK BOX,AGED GLASS SILVER T-LIGHT HOLDER,AIRLINE BAG VINTAGE JET SET BROWN,AIRLINE BAG VINTAGE JET SET RED,AIRLINE BAG VINTAGE JET SET WHITE,AIRLINE BAG VINTAGE TOKYO 78,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ALPHABET HEARTS STICKER SHEET,ANTIQUE ALL GLASS CANDLESTICK,ANTIQUE GLASS DRESSING TABLE POT,ANTIQUE GLASS PEDESTAL BOWL,ANTIQUE SILVER TEA GLASS ENGRAVED,APPLE BATH SPONGE,AREA PATROLLED METAL SIGN,ASS COLOUR GLOWING TIARAS,ASSORTED BOTTLE TOP MAGNETS,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOURS SILK FAN,ASSORTED EASTER DECORATIONS BELLS,...,TRAVEL CARD WALLET VINTAGE ROSE,TRAVEL CARD WALLET VINTAGE TICKET,TRAVEL SEWING KIT,"TUMBLER, NEW ENGLAND",TV DINNER TRAY DOLLY GIRL,UNION JACK FLAG PASSPORT COVER,URBAN BLACK RIBBONS,VICTORIAN SEWING BOX SMALL,VINTAGE BILLBOARD LOVE/HATE MUG,VINTAGE BILLBOARD TEA MUG,VINTAGE CARAVAN GIFT WRAP,VINTAGE CARAVAN GREETING CARD,VINTAGE CREAM 3 BASKET CAKE STAND,VINTAGE HEADS AND TAILS CARD GAME,VINTAGE SEASIDE JIGSAW PUZZLES,VINTAGE SNAKES & LADDERS,VINTAGE UNION JACK CUSHION COVER,WAKE UP COCKEREL TILE HOOK,WATERING CAN BLUE ELEPHANT,WATERING CAN GREEN DINOSAUR,WATERING CAN PINK BUNNY,WELCOME WOODEN BLOCK LETTERS,WHEELBARROW FOR CHILDREN,WHITE BROCANTE SOAP DISH,WHITE HANGING HEART T-LIGHT HOLDER,WHITE LOVEBIRD LANTERN,WICKER WREATH LARGE,WOOD BLACK BOARD ANT WHITE FINISH,WOOD STAMP SET HAPPY BIRTHDAY,WOODEN BOX OF DOMINOES,WOODEN CROQUET GARDEN SET,WOODEN HAPPY BIRTHDAY GARLAND,WOODEN PICTURE FRAME WHITE FINISH,WOODEN ROUNDERS GARDEN SET,WOODEN SKITTLES GARDEN SET,WOODLAND HEIGHT CHART STICKERS,WOODLAND STICKERS,WOODLAND CHARLOTTE BAG,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND PARTY BAG + STICKER SET,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP ALPHABET DESIGN,WRAP CAROUSEL,WRAP DOILEY DESIGN,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP VINTAGE PETALS DESIGN,ZINC METAL HEART DECORATION
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,24.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [7]:
def encode_units(x):
  if x<=0:
    return 0
  if x>=0:
    return 1
basket_sets= basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE WOODLAND,3D HEARTS HONEYCOMB PAPER GARLAND,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,60 TEATIME FAIRY CAKE CASES,72 SWEETHEART FAIRY CAKE CASES,ABC TREASURE BOOK BOX,AGED GLASS SILVER T-LIGHT HOLDER,AIRLINE BAG VINTAGE JET SET BROWN,AIRLINE BAG VINTAGE JET SET RED,AIRLINE BAG VINTAGE JET SET WHITE,AIRLINE BAG VINTAGE TOKYO 78,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ALPHABET HEARTS STICKER SHEET,ANTIQUE ALL GLASS CANDLESTICK,ANTIQUE GLASS DRESSING TABLE POT,ANTIQUE GLASS PEDESTAL BOWL,ANTIQUE SILVER TEA GLASS ENGRAVED,APPLE BATH SPONGE,AREA PATROLLED METAL SIGN,ASS COLOUR GLOWING TIARAS,ASSORTED BOTTLE TOP MAGNETS,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOURS SILK FAN,ASSORTED EASTER DECORATIONS BELLS,...,TRAVEL CARD WALLET VINTAGE ROSE,TRAVEL CARD WALLET VINTAGE TICKET,TRAVEL SEWING KIT,"TUMBLER, NEW ENGLAND",TV DINNER TRAY DOLLY GIRL,UNION JACK FLAG PASSPORT COVER,URBAN BLACK RIBBONS,VICTORIAN SEWING BOX SMALL,VINTAGE BILLBOARD LOVE/HATE MUG,VINTAGE BILLBOARD TEA MUG,VINTAGE CARAVAN GIFT WRAP,VINTAGE CARAVAN GREETING CARD,VINTAGE CREAM 3 BASKET CAKE STAND,VINTAGE HEADS AND TAILS CARD GAME,VINTAGE SEASIDE JIGSAW PUZZLES,VINTAGE SNAKES & LADDERS,VINTAGE UNION JACK CUSHION COVER,WAKE UP COCKEREL TILE HOOK,WATERING CAN BLUE ELEPHANT,WATERING CAN GREEN DINOSAUR,WATERING CAN PINK BUNNY,WELCOME WOODEN BLOCK LETTERS,WHEELBARROW FOR CHILDREN,WHITE BROCANTE SOAP DISH,WHITE HANGING HEART T-LIGHT HOLDER,WHITE LOVEBIRD LANTERN,WICKER WREATH LARGE,WOOD BLACK BOARD ANT WHITE FINISH,WOOD STAMP SET HAPPY BIRTHDAY,WOODEN BOX OF DOMINOES,WOODEN CROQUET GARDEN SET,WOODEN HAPPY BIRTHDAY GARLAND,WOODEN PICTURE FRAME WHITE FINISH,WOODEN ROUNDERS GARDEN SET,WOODEN SKITTLES GARDEN SET,WOODLAND HEIGHT CHART STICKERS,WOODLAND STICKERS,WOODLAND CHARLOTTE BAG,WOODLAND DESIGN COTTON TOTE BAG,WOODLAND PARTY BAG + STICKER SET,WORLD WAR 2 GLIDERS ASSTD DESIGNS,WRAP ALPHABET DESIGN,WRAP CAROUSEL,WRAP DOILEY DESIGN,WRAP GREEN PEARS,WRAP I LOVE LONDON,WRAP POPPIES DESIGN,WRAP RED APPLES,WRAP VINTAGE PETALS DESIGN,ZINC METAL HEART DECORATION
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,.

In [10]:
frequent_itemsets =apriori(basket_sets, min_support=0.07, use_colnames=True)
rules= association_rules(frequent_itemsets, metric='lift', min_threshold= 1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.102564,0.136752,0.076923,0.750000,5.484375,0.062897,3.452991,0.911111
1,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.136752,0.102564,0.076923,0.562500,5.484375,0.062897,2.051282,0.947195
2,(LUNCH BOX WITH CUTLERY RETROSPOT),(LUNCH BAG RED RETROSPOT),0.196581,0.170940,0.085470,0.434783,2.543478,0.051866,1.466798,0.755319
3,(LUNCH BAG RED RETROSPOT),(LUNCH BOX WITH CUTLERY RETROSPOT),0.170940,0.196581,0.085470,0.500000,2.543478,0.051866,1.606838,0.731959
4,(LUNCH BAG WOODLAND),(LUNCH BAG SPACEBOY DESIGN),0.128205,0.111111,0.085470,0.666667,6.000000,0.071225,2.666667,0.955882


In [12]:
rules[ (rules['lift'] >= 4) & (rules['confidence'] >= 0.8)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
38,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER CUPS),0.170940,0.205128,0.153846,0.900000,4.387500,0.118782,7.948718,0.931271
40,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER PLATES),0.170940,0.188034,0.145299,0.850000,4.520455,0.113157,5.413105,0.939357
42,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.188034,0.205128,0.179487,0.954545,4.653409,0.140916,17.487179,0.966917
43,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.205128,0.188034,0.179487,0.875000,4.653409,0.140916,6.495726,0.987711
50,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.145299,0.205128,0.145299,1.000000,4.875000,0.115494,inf,0.930000
51,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.153846,0.188034,0.145299,0.944444,5.022727,0.116371,14.615385,0.946524
52,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.179487,0.170940,0.145299,0.809524,4.735714,0.114618,4.352564,0.961397
53,(SET/20 RED RETROSPOT PAPER NAPKINS),"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",0.170940,0.179487,0.145299,0.850000,4.735714,0.114618,5.470085,0.951486
